In [1]:
import jax.random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%load_ext autoreload

%cd /mnt/c/Users/Habush/PycharmProjects/bnn_bg
from s3.helper_functions import *
from s3.mcmc_functions import *
from core.spike_slab import *

/mnt/c/Users/Habush/PycharmProjects/bnn_bg


2024-01-05 14:05:27.854130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_PATH = '/mnt/c/Users/Habush/PycharmProjects/bnn_bg/data'
EXP_PATH = '/mnt/c/Users/Habush/PycharmProjects/bnn_bg/experiments'
CCLE_PATH = f"{DATA_PATH}/ccle"

In [3]:
string_ppi_df = pd.read_csv(f"{DATA_PATH}/string_ppi.csv")

In [23]:
# Load data
ccle_data_df = pd.read_csv(f"{CCLE_PATH}/CCLE_expression.csv", index_col=0)

In [27]:
model_info_df = pd.read_csv(f"{CCLE_PATH}/model_info.csv", index_col=0)
drug_resonse_df = pd.read_csv(f"{DATA_PATH}/GDSC2_fitted_dose_response_24Jul22.csv", index_col="SANGER_MODEL_ID")

In [28]:
#Keep only cell lines we have drug response data for
print(f"Before filtering: {drug_resonse_df.shape}")
drug_resonse_df = drug_resonse_df[drug_resonse_df.index.isin(model_info_df["SangerModelID"])]
print(f"After filtering: {drug_resonse_df.shape}")

Before filtering: (242036, 18)
After filtering: (241578, 18)


In [29]:
#categorize cell lines by cancer type
model_info_df["OncotreeCode"].value_counts()

GB         97
MEL        94
PAAD       94
LUAD       90
SCLC       81
           ..
HCCIHCH     1
GBC         1
CEGCC       1
NSCLCPD     1
TPLL        1
Name: OncotreeCode, Length: 199, dtype: int64

In [96]:
import haiku as hk
import jax.numpy as jnp
import tensorflow as tf

def softmax_cross_entropy(logits, labels):
  one_hot = jax.nn.one_hot(labels, logits.shape[-1])
  return -jnp.sum(jax.nn.log_softmax(logits) * one_hot, axis=-1)

def loss_fn(images, labels):
  mlp = hk.Sequential([
      hk.Linear(300), jax.nn.relu,
      hk.Linear(100), jax.nn.relu,
      hk.Linear(10),
  ])
  logits = mlp(images)
  return jnp.mean(softmax_cross_entropy(logits, labels))

loss_fn_t = hk.transform(loss_fn)
loss_fn_t = hk.without_apply_rng(loss_fn_t)
rng = jax.random.PRNGKey(42)
x = jax.random.normal(rng, (100, 784))
label = jax.random.randint(rng, (100,), 0, 2)

input_dataset = iter(tf.data.Dataset.from_tensor_slices((x, label)))

dummy_images, dummy_labels = x[0], label[0]
params = loss_fn_t.init(rng, dummy_images, dummy_labels)
# params
# def update_rule(param, update):
#   return param - 0.01 * update
# 
# for images, labels in input_dataset:
#   grads = jax.grad(loss_fn_t.apply)(params, images, labels)
#   params = jax.tree_util.tree_map(update_rule, params, grads)

In [89]:
def log_gaussian(x):
    return jnp.sum(tfd.Normal(0, 1).log_prob(x))

jax.tree_structure(params)

/tmp/ipykernel_3664/2611997145.py:4: DeprecationWarning: jax.tree_structure is deprecated: use jax.tree_util.tree_structure.
  jax.tree_structure(params)


PyTreeDef({'linear': {'b': *, 'w': *}, 'linear_1': {'b': *, 'w': *}, 'linear_2': {'b': *, 'w': *}})

In [90]:
len(jax.tree_leaves(params))

/tmp/ipykernel_3664/1714611174.py:1: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree_util.tree_leaves.
  len(jax.tree_leaves(params))


6

In [91]:
def get_first_elem_in_sharded_tree(tree):
  return jax.tree_map(lambda p: p[0], tree)

In [92]:
sample_tree = {'a': jnp.array([1, 2, 3]), 'b': jnp.array([4, 5, 6])}
get_first_elem_in_sharded_tree(sample_tree)

{'a': Array(1, dtype=int32), 'b': Array(4, dtype=int32)}

In [162]:
%autoreload
from utils import tree_utils
import operator
def make_gaussian_log_prior1(weight_decay, temperature):
  """Returns the Gaussian log-density and delta given weight decay."""

  def log_prior(params):
    """Computes the Gaussian prior log-density."""
    # ToDo izmailovpavel: make temperature treatment the same as in gaussian
    # likelihood function.
    sizes = [p.size for p in jax.tree_leaves(params)]
    print(f"sizes: {sizes}")
    n_params = sum([p.size for p in jax.tree_leaves(params)])
    log_prob = -(0.5 * tree_utils.tree_dot(params, params) * weight_decay +
                 0.5 * n_params * jnp.log(weight_decay / (2 * jnp.pi)))
    return log_prob / temperature

  return log_prior

def make_gaussian_log_prior2(weight_decay, temperature):
  """Returns the Gaussian log-density and delta given weight decay."""

  def log_prior(params):
    """Computes the Gaussian prior log-density."""
    # ToDo izmailovpavel: make temperature treatment the same as in gaussian
    # likelihood function.
    
    def fn(x):
        dist = tfd.MultivariateNormalDiag(jnp.zeros_like(x), jnp.ones_like(x)*weight_decay)
        print(f"dist: {dist}")
        return jnp.sum(dist.unnormalized_log_prob(x))
    
    log_prob = sum(jax.tree_leaves(jax.tree_map(fn, params)))
    return log_prob / temperature

  return log_prior

In [163]:
log_prior1 = make_gaussian_log_prior1(1, 1)
log_prior1(params)

sizes: [300, 235200, 100, 30000, 10, 1000]


/tmp/ipykernel_3664/708325981.py:11: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree_util.tree_leaves.
  sizes = [p.size for p in jax.tree_leaves(params)]
/tmp/ipykernel_3664/708325981.py:13: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree_util.tree_leaves.
  n_params = sum([p.size for p in jax.tree_leaves(params)])


TypeError: add got incompatible shapes for broadcasting: (300, 300), (100, 100).

In [156]:
log_prior2 = make_gaussian_log_prior2(1, 1)
log_prior2(params)

/tmp/ipykernel_3664/928529965.py:30: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree_util.tree_leaves.
  log_prob = sum(jax.tree_leaves(jax.tree_map(fn, params)))


dist: tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[], event_shape=[300], dtype=float32)
dist: tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[784], event_shape=[300], dtype=float32)
dist: tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[], event_shape=[100], dtype=float32)
dist: tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[300], event_shape=[100], dtype=float32)
dist: tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[], event_shape=[10], dtype=float32)
dist: tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[100], event_shape=[10], dtype=float32)


Array(-245157.42, dtype=float32)

In [34]:
jax.tree_leaves(params)

/tmp/ipykernel_3664/137367794.py:1: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree_util.tree_leaves.
  jax.tree_leaves(params)


[Array([1., 2., 3.], dtype=float32),
 Array([4., 5., 6.], dtype=float32),
 Array([1., 2., 3.], dtype=float32)]

In [75]:
params

{'a': Array([1., 2., 3.], dtype=float32),
 'b': Array([4., 5., 6.], dtype=float32),
 'c': Array([1., 2., 3.], dtype=float32)}

In [161]:
def tree_dot(a, b):
  # return sum([jnp.sum(e1 * e2) for e1, e2 in
              # zip(jax.tree_util.tree_leaves(a), jax.tree_util.tree_leaves(b))])
    for e1, e2 in zip(jax.tree_util.tree_leaves(a), jax.tree_util.tree_leaves(b)):
        print(f"e1: {e1.shape}, e2: {e2.shape}")
# tfd.Normal(0, 1).unnormalized_log_prob(tree_dot(params, params))/sum([p.size for p in jax.tree_leaves(params)])

tree_dot(params, params)

e1: (300,), e2: (300,)
e1: (784, 300), e2: (784, 300)
e1: (100,), e2: (100,)
e1: (300, 100), e2: (300, 100)
e1: (10,), e2: (10,)
e1: (100, 10), e2: (100, 10)


In [174]:
x = jnp.array([0.5, 0.5, 0.5])
sigma2, tau1, tau0 = 1., 5., 1e-2
slab_lp_fn = lambda t: tfd.MultivariateNormalDiag(0, jnp.ones_like(t)*(tau1**2)*sigma2).log_prob(t)
spike_lp_fn = lambda t: tfd.MultivariateNormalDiag(0, jnp.ones_like(t)*(tau0**2)*sigma2).log_prob(t)

slap_log_prob = slab_lp_fn(x)
spike_log_prob = spike_lp_fn(x)
print(f"slap_log_prob: {slap_log_prob}, spike_log_prob: {spike_log_prob}")

slap_log_prob: -12.414043426513672, spike_log_prob: -37499976.0
